In [2]:
import pandas as pd
import numpy as np

In [3]:
precipitation = pd.read_csv("data/processed/precipitation.csv")
moisture = pd.read_csv("data/processed/moisture.csv")
pression = pd.read_csv("data/processed/pression.csv")
temperature = pd.read_csv("data/processed/temperature.csv")
wind = pd.read_csv("data/processed/wind_vectors.csv")

In [4]:
stations = pd.read_csv("valais_special/data/station_list.csv", encoding="Windows 1252", delimiter=";")
valais_stations = stations[stations['Canton']=="VS"]


In [5]:
precipitation_valais = precipitation[precipitation['station'].isin(valais_stations['Abr.'])]
moisture_valais = moisture[moisture['station'].isin(valais_stations['Abr.'])]
pression_valais = pression[pression['station'].isin(valais_stations['Abr.'])]
temperature_valais = temperature[temperature['station'].isin(valais_stations['Abr.'])]
wind_valais = wind[wind['station'].isin(valais_stations['Abr.'])]

In [6]:
precipitation_valais.index = pd.to_datetime(precipitation_valais['time'], format='%Y%m%d%H%M')

In [7]:
precipitation_valais.groupby('station').head(1)

,station,time,precip
time,,,
2019-01-01 00:00:00,VSANZ,201901010000,0.0
2019-01-01 00:00:00,VSARO,201901010000,0.0
2019-01-01 00:00:00,VSBAS,201901010000,0.0
2019-01-01 00:00:00,VSGDX,201901010000,0.0
2019-01-01 00:00:00,BIN,201901010000,0.0
2019-01-01 00:00:00,BLA,201901010000,0.0
2019-01-01 00:00:00,VSBSP,201901010000,0.0
2019-01-01 00:00:00,VSBRI,201901010000,0.0
2019-01-01 00:00:00,BRI,201901010000,0.0


In [31]:
precip_wide_vs = precipitation_valais.pivot(columns='station', values='precip').reset_index()
print(len(precip_wide_vs))

262944


In [32]:
cleaned_data = precip_wide_vs.dropna()
cleaned_data
# Get indices of remaining valid rows
valid_indices = cleaned_data.index

# Find contiguous segments
gaps = np.diff(valid_indices) > 1  # True if there's a gap
segment_breaks = np.where(gaps)[0]  # Indices where gaps occur

# Split into segments
segment_lengths = np.diff(np.hstack(([valid_indices[0] - 1], valid_indices[segment_breaks], [valid_indices[-1]])))

# Compute number of rolling windows per segment
window_size = 72
num_rolling_windows = sum(max(0, seg_len - (window_size - 1)) for seg_len in segment_lengths)

print(f"Number of valid rolling windows of size {window_size}: {num_rolling_windows}")

Number of valid rolling windows of size 72: 246815
